In [33]:
import numpy as np

LIGHT_SPEED = 299792.458

In [34]:
def calculate_jacobian(X):
    jacobian = np.zeros((4,4))
    C = LIGHT_SPEED
    x = X[0,0]
    y = X[1,0]
    z = X[2,0]
    D = X[3,0]
    jacobian[0,0] = 2 * (x - 15600)
    jacobian[1,0] = 2 * (x - 18760)
    jacobian[2,0] = 2 * (x - 17610)
    jacobian[3,0] = 2 * (x - 19170)
   
    jacobian[0,1] = 2 * (y - 7540)
    jacobian[1,1] = 2 * (y - 2750)
    jacobian[2,1] = 2 * (y - 14630)
    jacobian[3,1] = 2 * (y - 610)
   
    jacobian[0,2] = 2 * (z - 20140)
    jacobian[1,2] = 2 * (z - 18610)
    jacobian[2,2] = 2 * (z - 13480)
    jacobian[3,2] = 2 * (z - 18390)
   
    jacobian[0,3] = 2 * (C**2) * 0.07074 - 2 * (C**2) * D
    jacobian[1,3] = 2 * (C**2) * 0.07220 - 2 * (C**2) * D
    jacobian[2,3] = 2 * (C**2) * 0.07690 - 2 * (C**2) * D
    jacobian[3,3] = 2 * (C**2) * 0.07242 - 2 * (C**2) * D
 
    return jacobian

In [35]:
def functions(X, locations: np.ndarray):
    C = LIGHT_SPEED
    x = X[0,0]
    y = X[1,0]
    z = X[2,0]
    D = X[3,0]
    Fa = np.array([[0],[0],[0],[0]])
    Fa[0,0] = (x - locations[0][0])**2 + (y - locations[0][1])**2 + (z - locations[0][2])**2 - (C * (0.07074 - D))**2
    Fa[1,0] = (x - locations[1][0])**2 + (y - locations[1][1])**2 + (z - locations[1][2])**2 - (C * (0.07220 - D))**2
    Fa[2,0] = (x - locations[2][0])**2 + (y - locations[2][1])**2 + (z - locations[2][2])**2 - (C * (0.07690 - D))**2
    Fa[3,0] = (x - locations[3][0])**2 + (y - locations[3][1])**2 + (z - locations[3][2])**2 - (C * (0.07242 - D))**2
    return Fa

In [36]:
def calculate_location(locations: np.ndarray):
    X = np.array([[0], [0], [6370], [0]])
    for _ in range(30):
        F = functions(X, locations)
        J = calculate_jacobian(X)
        H = np.linalg.solve(J, F)
        X = X - H        
    print(X)

In [37]:
def main():
    locations = np.array([
        [15600, 6540, 20140], 
        [18700, 2750, 18610], 
        [17610, 14630, 13480], 
        [19170, 610, 18390]
    ])

    calculate_location(locations)

main()

[[ 1.26760661e+03]
 [-1.69380053e+02]
 [ 5.37507603e+03]
 [-1.45457260e-03]]
